# Bag-of-Words Classifier Pipeline

In [1]:
import os

import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# Custom functions 
from utils import print_gridsearch_results, test_on_estimator

RANDOM_STATE = 123

## Data prep

In [2]:
data_dir = 'data_reviews'
x_train_df = pd.read_csv(os.path.join(data_dir, 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))

# Where to output predictions on the test_set
x_test_df = pd.read_csv(os.path.join(data_dir, 'x_test.csv'))
output_dir = 'predictions'

print("Shape of data\n---------------")
print(f"x_train_df shape: {x_train_df.shape} ")
print(f"x_test_df shape: {x_test_df.shape} ")
print(f"y_train_df shape: {y_train_df.shape} ")

# Get the text as a list of strings
x_train_text = x_train_df['text'].values
x_test_text = x_test_df['text'].values
y_train = y_train_df['is_positive_sentiment'].values

Shape of data
---------------
x_train_df shape: (2400, 2) 
x_test_df shape: (600, 2) 
y_train_df shape: (2400, 1) 


## Dataset Exploration

In [3]:
num_train_samples = x_train_text.shape[0]
num_positive_train_samples = np.count_nonzero(y_train == 1)
fraction_positive_train = float(num_positive_train_samples) / float(num_train_samples)


print(f"Total number of training samples = {num_train_samples}")
print(f"Fraction positive training samples = {fraction_positive_train}")


Total number of training samples = 2400
Fraction positive training samples = 0.5


## Define a stable skf CV-splitter

In [8]:
cv_splitter = sklearn.model_selection.StratifiedKFold(n_splits=7)

## Basic comparison of two vectorizers - one with counts and one using tfidf


In [12]:
count_vectorizer = CountVectorizer(stop_words='english')
#  Parameters for grid search
count_parameters = {
    'count__min_df': np.arange(1, 3),
#     'count__max_df': (0.05, 0.5),
    'count__ngram_range': [(1, 1), (1, 2)],
}
count_pipeline_tuple = ("count", count_vectorizer)
# count_vectorizer.vocabulary_

In [13]:
tf_vectorizer = TfidfVectorizer(stop_words='english')
tf_vectorizer.fit_transform(x_train_text)
#  Parameters for grid search
tf_parameters = { 
    'tf__min_df': np.arange(1,3),
#     'tf__max_df': (0.05, 0.5),
    'tf__ngram_range': [(1, 1), (1, 2)],
}
tf_pipeline_tuple = ("tf", tf_vectorizer)
# tf_vectorizer.vocabulary_

## Random Forest: Hyperparameter search and CV

In [14]:
rf_classifier = RandomForestClassifier(random_state=RANDOM_STATE)
# Random Forest hyperparameter grid
rf_parameters = {
    "rf__n_estimators": [100, 200, 300],
    "rf__max_depth": [5, 10, 20],
}
rf_pipeline_tuple = ('rf', rf_classifier)

### Fit on the tfidf_vectorizer

In [18]:
tf_rf_pipeline = Pipeline([
    tf_pipeline_tuple,
    rf_pipeline_tuple,
 ])
tf_rf_full_grid = { 
    **tf_parameters,
    **rf_parameters
}


tf_rf_grid_searcher = GridSearchCV(
    tf_rf_pipeline, 
    tf_rf_full_grid, 
    cv=cv_splitter, 
    n_jobs=4, 
    verbose=3,
    scoring='balanced_accuracy',
    return_train_score=True
)
tf_rf_grid_searcher.fit(x_train_text, y_train)

Fitting 7 folds for each of 36 candidates, totalling 252 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    2.6s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   18.7s
[Parallel(n_jobs=4)]: Done 252 out of 252 | elapsed:   51.0s finished


GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('tf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('rf',
                                        RandomForestClassifier(random_state=123))]),
             n_jobs=4,
             param_grid={'rf__max_depth': [5, 10, 20],
                         'rf__n_estimators': [100, 200, 300],
                         'tf__min_df': array([1, 2]),
                         'tf__ngram_range': [(1, 1), (1, 2)]},
             return_train_score=True, scoring='balanced_accuracy', verbose=3)

In [19]:
print_gridsearch_results(tf_rf_grid_searcher, list(tf_rf_full_grid.keys()))
# tf_rf_grid_searcher.cv_results_.keys()

Dataframe has shape: (36, 28)
Number of trials used in grid search:  36


,param_tf__min_df,param_tf__ngram_range,param_rf__n_estimators,param_rf__max_depth,mean_train_score,mean_test_score,mean_fit_time,rank_test_score
9,1,"(1, 2)",300,5,0.845262,0.757652,0.777539,1
12,1,"(1, 1)",100,10,0.834093,0.755083,0.290679,2
24,1,"(1, 1)",100,20,0.858679,0.755044,0.355858,3
21,1,"(1, 2)",300,10,0.867630,0.754296,1.012810,4
29,1,"(1, 2)",200,20,0.884648,0.753835,0.917830,5
17,1,"(1, 2)",200,10,0.861448,0.753463,0.630203,6
25,1,"(1, 2)",100,20,0.879924,0.752586,0.419309,7
5,1,"(1, 2)",200,5,0.836581,0.751834,0.535864,8
33,1,"(1, 2)",300,20,0.887912,0.751328,1.302007,9
27,2,"(1, 2)",100,20,0.836110,0.750899,0.357823,10


In [16]:
# print_gridsearch_results(tf_rf_grid_searcher, list(tf_rf_full_grid.keys()))
# tf_rf_grid_searcher.cv_results_.keys()

Dataframe has shape: (9, 26)
Number of trials used in grid search:  9


,param_rf__n_estimators,param_rf__max_depth,mean_train_score,mean_test_score,mean_fit_time,rank_test_score
3,100,10,0.834093,0.755083,0.285056,1
6,100,20,0.858679,0.755044,0.401576,2
8,300,20,0.864164,0.750471,0.972810,3
7,200,20,0.859024,0.750083,0.797344,4
5,300,10,0.843605,0.749245,0.772787,5
4,200,10,0.837285,0.748434,0.546363,6
2,300,5,0.825335,0.746369,0.711360,7
1,200,5,0.820611,0.745546,0.503189,8
0,100,5,0.806241,0.736796,0.235691,9


In [20]:
tf_rf_grid_searcher.best_params_

{'rf__max_depth': 5,
 'rf__n_estimators': 300,
 'tf__min_df': 1,
 'tf__ngram_range': (1, 2)}

### Fit on CountVector

In [23]:
count_rf_pipeline = Pipeline([
    count_pipeline_tuple,
    rf_pipeline_tuple,
 ])
count_rf_full_grid = { 
    **count_parameters,
    **rf_parameters
}


count_rf_grid_searcher = GridSearchCV(
    count_rf_pipeline, 
    count_rf_full_grid, 
    cv=cv_splitter, 
    n_jobs=3, 
    verbose=3,
    return_train_score=True,
    scoring='balanced_accuracy'
)
count_rf_grid_searcher.fit(x_train_text, y_train)

Fitting 7 folds for each of 36 candidates, totalling 252 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:    4.3s
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed:   25.0s
[Parallel(n_jobs=3)]: Done 252 out of 252 | elapsed:   49.6s finished


GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('count',
                                        CountVectorizer(stop_words='english')),
                                       ('rf',
                                        RandomForestClassifier(random_state=123))]),
             n_jobs=3,
             param_grid={'count__min_df': array([1, 2]),
                         'count__ngram_range': [(1, 1), (1, 2)],
                         'rf__max_depth': [5, 10, 20],
                         'rf__n_estimators': [100, 200, 300]},
             return_train_score=True, scoring='balanced_accuracy', verbose=3)

In [24]:
print_gridsearch_results(count_rf_grid_searcher, list(count_rf_full_grid.keys()))

Dataframe has shape: (36, 28)
Number of trials used in grid search:  36


,param_count__min_df,param_count__ngram_range,param_rf__n_estimators,param_rf__max_depth,mean_train_score,mean_test_score,mean_fit_time,rank_test_score
17,1,"(1, 2)",300,20,0.876524,0.767143,0.837929,1
16,1,"(1, 2)",200,20,0.873676,0.766728,0.576524,2
15,1,"(1, 2)",100,20,0.873816,0.765467,0.318968,3
14,1,"(1, 2)",300,10,0.857908,0.763862,0.676434,4
13,1,"(1, 2)",200,10,0.855199,0.763024,0.459837,5
6,1,"(1, 1)",100,20,0.851113,0.761684,0.289146,6
11,1,"(1, 2)",300,5,0.836514,0.759296,0.605566,7
8,1,"(1, 1)",300,20,0.855556,0.759190,0.809340,8
7,1,"(1, 1)",200,20,0.852292,0.758772,0.567307,9
5,1,"(1, 1)",300,10,0.833818,0.758376,0.640886,10


In [100]:
print_gridsearch_results(count_rf_grid_searcher, list(count_rf_full_grid.keys()))

Dataframe has shape: (72, 31)
Number of trials used in grid search:  72


,param_count__min_df,param_count__max_df,param_count__ngram_range,param_rf__n_estimators,param_rf__max_depth,mean_train_score,mean_test_score,mean_fit_time,rank_test_score
42,1,0.5,"(1, 1)",100,20,0.853810,0.765833,0.290650,1
52,1,0.5,"(1, 2)",200,20,0.876310,0.764583,0.784306,2
51,1,0.5,"(1, 2)",100,20,0.877679,0.762083,0.533752,3
49,1,0.5,"(1, 2)",200,10,0.851012,0.762083,0.684137,4
48,1,0.5,"(1, 2)",100,10,0.845714,0.760833,0.324446,5
...,...,...,...,...,...,...,...,...,...
9,1,0.05,"(1, 2)",100,5,0.792083,0.707500,0.254434,68
21,2,0.05,"(1, 1)",100,10,0.802917,0.705833,0.219317,69
27,2,0.05,"(1, 2)",100,5,0.773571,0.702500,0.226318,70
0,1,0.05,"(1, 1)",100,5,0.781369,0.694583,0.235885,71


In [25]:
count_rf_grid_searcher.best_params_

{'count__min_df': 1,
 'count__ngram_range': (1, 2),
 'rf__max_depth': 20,
 'rf__n_estimators': 300}

## Generate predictions using best Random Forest and Count Vectorizer

In [102]:
count_rf_filename = os.path.join(output_dir, 'bow_count_rf_yproba1_test.txt')
test_on_estimator(count_rf_grid_searcher.best_estimator_, x_test_text, count_rf_filename)
tf_rf_filename = os.path.join(output_dir, 'bow_tf_rf_yproba1_test.txt')
test_on_estimator(tf_rf_grid_searcher.best_estimator_, x_test_text, tf_rf_filename)

# Gradient Boosted Tree: Hyperparameter search

**NOTE** Only re-run when necessary. The total run-time for rebuilding these classifiers is roughly 15-30 minutes.

In [29]:
gbtree_classifier = GradientBoostingClassifier(n_estimators=100, max_depth=5, min_samples_leaf=1, random_state=RANDOM_STATE)
# GradientBoosting-Tree hyperparameter grid
gbtree_parameters = {
    "gbtree__n_estimators": [100, 200, 300],
    "gbtree__max_depth": [5, 10, 20],
    "gbtree__min_samples_leaf": [1, 3, 5],
}
gbtree_pipeline_tuple = ('gbtree', gbtree_classifier)


### Fit on the tfidf_vectorizer

In [31]:
tf_gbtree_pipeline = Pipeline([
    tf_pipeline_tuple,
    gbtree_pipeline_tuple,
 ])
tf_gbtree_full_grid = { 
    **tf_parameters,
    **gbtree_parameters
}


tf_gbtree_grid_searcher = GridSearchCV(
    tf_gbtree_pipeline, 
    tf_gbtree_full_grid, 
    cv=cv_splitter, 
    n_jobs=3, 
    verbose=3,
    scoring='balanced_accuracy',
    return_train_score=True
)
tf_gbtree_grid_searcher.fit(x_train_text, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   10.0s
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed:  1.4min
[Parallel(n_jobs=3)]: Done 282 tasks      | elapsed:  2.7min
[Parallel(n_jobs=3)]: Done 506 tasks      | elapsed:  6.4min
[Parallel(n_jobs=3)]: Done 794 tasks      | elapsed: 11.6min
[Parallel(n_jobs=3)]: Done 1080 out of 1080 | elapsed: 20.6min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('tf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('gbtree',
                                        GradientBoostingClassifier(max_depth=5,
                                                                   random_state=123))]),
             n_jobs=3,
             param_grid={'gbtree__max_depth': [5, 10, 20],
                         'gbtree__min_samples_leaf': [1, 3, 5],
                         'gbtree__n_estimators': [100, 200, 300],
                         'tf__max_df': (0.05, 0.5), 'tf__min_df': array([1, 2]),
                         'tf__ngram_range': [(1, 1), (1, 2)]},
             return_train_score=True, scoring='balanced_accuracy', verbose=3)

In [32]:
print_gridsearch_results(tf_gbtree_grid_searcher, list(tf_gbtree_full_grid.keys()))

Dataframe has shape: (216, 26)
Number of trials used in grid search:  216


,param_tf__min_df,param_tf__max_df,param_tf__ngram_range,param_gbtree__n_estimators,param_gbtree__max_depth,param_gbtree__min_samples_leaf,mean_train_score,mean_test_score,mean_fit_time,rank_test_score
20,1,0.5,"(1, 1)",300,5,1,0.941979,0.776667,2.447076,1
12,1,0.5,"(1, 1)",200,5,1,0.923750,0.771667,1.560614,2
92,1,0.5,"(1, 1)",300,10,1,0.996146,0.770833,4.124050,3
22,2,0.5,"(1, 1)",300,5,1,0.947604,0.769583,1.807707,4
84,1,0.5,"(1, 1)",200,10,1,0.995937,0.769583,2.854647,5
...,...,...,...,...,...,...,...,...,...,...
27,2,0.05,"(1, 2)",100,5,3,0.819583,0.683333,0.581437,212
209,1,0.05,"(1, 2)",300,20,5,0.940521,0.683333,11.528023,212
48,1,0.05,"(1, 1)",100,5,5,0.797708,0.680833,0.613226,214
211,2,0.05,"(1, 2)",300,20,5,0.940417,0.680417,4.091111,215


In [34]:
tf_gbtree_grid_searcher.best_params_

{'gbtree__max_depth': 5,
 'gbtree__min_samples_leaf': 1,
 'gbtree__n_estimators': 300,
 'tf__max_df': 0.5,
 'tf__min_df': 1,
 'tf__ngram_range': (1, 1)}

## GBTree: Generate Predictions

In [37]:
gbtree_filename = os.path.join(output_dir, 'bow_tf_gbtree_yproba1_test.txt')
test_on_estimator(tf_gbtree_grid_searcher.best_estimator_, x_test_text, gbtree_filename)

## TBD: Figures

In [ ]:
fig, loss_ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 4), sharex=True, sharey=True)

for run_id in range(n_runs):
    tr_label = 'train log loss' if run_id == 0 else ''
    te_label = 'test log loss' if run_id == 0 else ''
    
    loss_ax.plot(np.log2(size_list), tr_loss_arr[:,run_id], 'bd', label=tr_label)
    loss_ax.plot(np.log2(size_list), te_loss_arr[:,run_id], 'rd', label=te_label)

loss_ax.set_xticks(np.log2(size_list));
loss_ax.set_xticklabels(size_list);
loss_ax.xaxis.grid(False);

loss_ax.set_ylim([0, 0.8]); # Don't touch this please
loss_ax.set_yticks(np.arange(0, 0.8, 0.1));
loss_ax.set_title("Log Loss vs Size")
loss_ax.set_ylabel('log loss');
loss_ax.set_xlabel('size');
loss_ax.legend(loc='upper right');


## L1-Logistic Regression: Hyperparameter Search

In [58]:
lasso = sklearn.linear_model.LogisticRegression(penalty='l1', solver='saga', random_state=RANDOM_STATE)
# logit_lasso hyperparameter grid
lasso_parameters = {
     'lasso__C': np.logspace(-4, 4, 9),
     'lasso__max_iter': [20, 40], # sneaky way to do "early stopping" 
}
lasso_pipeline_tuple = ('lasso', lasso)


### Fit on the tfidf_vectorizer

In [59]:
count_lasso_pipeline = Pipeline([
    count_pipeline_tuple,
    lasso_pipeline_tuple,
 ])
count_lasso_full_grid = { 
    **count_parameters,
    **lasso_parameters
}


count_lasso_grid_searcher = GridSearchCV(
    count_lasso_pipeline, 
    count_lasso_full_grid, 
    cv=cv_splitter, 
    n_jobs=3, 
    verbose=3,
    scoring='balanced_accuracy',
    return_train_score=True
)
count_lasso_grid_searcher.fit(x_train_text, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:    2.4s
[Parallel(n_jobs=3)]: Done 143 tasks      | elapsed:   26.3s
[Parallel(n_jobs=3)]: Done 303 tasks      | elapsed:  1.5min
[Parallel(n_jobs=3)]: Done 545 tasks      | elapsed:  2.9min
[Parallel(n_jobs=3)]: Done 720 out of 720 | elapsed:  3.3min finished
/Users/dylanphelan/opt/miniconda3/envs/comp135_2020f_env/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('count',
                                        CountVectorizer(stop_words='english')),
                                       ('lasso',
                                        LogisticRegression(penalty='l1',
                                                           random_state=101,
                                                           solver='saga'))]),
             n_jobs=3,
             param_grid={'count__max_df': (0.05, 0.5),
                         'count__min_df': array([1, 2]),
                         'count__ngram_range': [(1, 1), (1, 2)],
                         'lasso__C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04]),
                         'lasso__max_iter': [20, 40]},
             return_train_score=True, scoring='balanced_accuracy', verbose=3)

In [60]:
print_gridsearch_results(count_lasso_grid_searcher, list(count_lasso_full_grid.keys()))

Dataframe has shape: (144, 25)
Number of trials used in grid search:  144


,param_count__min_df,param_count__max_df,param_count__ngram_range,param_lasso__C,param_lasso__max_iter,mean_train_score,mean_test_score,mean_fit_time,rank_test_score
84,1,0.5,"(1, 1)",100,20,0.967708,0.797083,0.914771,1
82,1,0.5,"(1, 1)",10,20,0.963646,0.797083,0.749147,1
88,1,0.5,"(1, 1)",10000,20,0.968021,0.796250,0.920900,3
86,1,0.5,"(1, 1)",1000,20,0.967917,0.796250,0.968059,3
104,1,0.5,"(1, 2)",1000,20,0.978958,0.795417,2.858857,5
...,...,...,...,...,...,...,...,...,...
109,2,0.5,"(1, 1)",0.0001,40,0.500000,0.500000,0.027523,97
110,2,0.5,"(1, 1)",0.001,20,0.500000,0.500000,0.029619,97
111,2,0.5,"(1, 1)",0.001,40,0.500000,0.500000,0.027353,97
113,2,0.5,"(1, 1)",0.01,40,0.500000,0.500000,0.030030,97


In [61]:
count_lasso_grid_searcher.best_params_

{'count__max_df': 0.5,
 'count__min_df': 1,
 'count__ngram_range': (1, 1),
 'lasso__C': 10.0,
 'lasso__max_iter': 20}

## L1-Logistic Regression: Generate Predictions

In [62]:
lasso_filename = os.path.join(output_dir, 'bow_count_lasso_yproba1_test.txt')
test_on_estimator(count_lasso_grid_searcher.best_estimator_, x_test_text, lasso_filename)

# Archive: Things We May Need 



In [39]:
# np.argwhere(count_rf_grid_searcher.cv_results_["rank_test_score"] == 1)
# count_rf_grid_searcher.cv_results_["mean_train_score"][29]

In [40]:
# random_forest_grid_searcher.fit(x_train_text_count, y_train)

In [41]:
# random_forest_grid_search_results_df = pd.DataFrame(random_forest_grid_searcher.cv_results_).copy()
# n_trials_grid_search = random_forest_grid_search_results_df.shape[0]

In [42]:
# param_keys = ['param_n_estimators', 'param_max_depth']

# # Rearrange row order so it is easy to skim
# random_forest_grid_search_results_df.sort_values(param_keys, inplace=True)
# random_forest_grid_search_results_df[param_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score']]

### Fit on TFIDF

In [43]:
# random_forest_grid_searcher.fit(x_train_text_tf, y_train)

In [44]:
# random_forest_grid_search_results_df = pd.DataFrame(random_forest_grid_searcher.cv_results_).copy()
# n_trials_grid_search = random_forest_grid_search_results_df.shape[0]

In [45]:
# param_keys = ['param_n_estimators', 'param_max_depth']

# Rearrange row order so it is easy to skim
# random_forest_grid_search_results_df.sort_values(param_keys, inplace=True)
# random_forest_grid_search_results_df[param_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score']]

In [56]:
# best_random_forest = random_forest_classifier.set_params(**random_forest_grid_searcher.best_params_)
# best_random_forest.fit(x_train_text_count, y_train)

In [47]:
# # Gradient Boosted Tree classifier with default values
# gbtree_classifier = GradientBoostingClassifier(n_estimators=100, max_depth=5, min_samples_leaf=1, random_state=100)

In [48]:
# # Gradient Boosted Tree hyperparameter grid
# gbtree_parameter_grid = dict(
#     n_estimators = [20, 50, 100],
#     max_depth = [5, 10],
#     #min_samples_leaf = [1, 3, 5],
#     #random_state=[101, 202],  # try two possible seeds to initialize parameters
#     random_state=[100],
#     )

In [49]:
# gbtree_grid_searcher = sklearn.model_selection.GridSearchCV(
#     gbtree_classifier,
#     gbtree_parameter_grid,
#     scoring='balanced_accuracy',
#     cv=cv_splitter,
#     return_train_score=True,
#     refit=False)

In [50]:
# gbtree_grid_searcher.fit(x_train_text_count, y_train)

In [51]:
# gbtree_grid_search_results_df = pd.DataFrame(gbtree_grid_searcher.cv_results_).copy()
# n_trials_grid_search = gbtree_grid_search_results_df.shape[0]

In [52]:
# param_keys = ['param_n_estimators', 'param_max_depth']

# # Rearrange row order so it is easy to skim
# gbtree_grid_search_results_df.sort_values(param_keys, inplace=True)

In [53]:
# gbtree_grid_search_results_df[param_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score']]

In [54]:
# best_gbtree = gbtree_classifier.set_params(**gbtree_grid_searcher.best_params_)
# best_gbtree.fit(x_train_text_count, y_train)

In [63]:
# best_gbtree_yhat_test = best_gbtree.predict(x_test_text_count)
# with open('bow_gbtree_countvectorizer_test_preds.txt', 'w') as f:
#     for pred in best_gbtree_yhat_test:
#         f.write(str(pred) + '\n')

In [64]:
# lasso = sklearn.linear_model.LogisticRegression(
#     penalty='l1', solver='saga', random_state=101)

In [65]:
# lasso_hyperparameter_grid_by_name = dict(
#     C=np.logspace(-4, 4, 9),
#     max_iter=[20, 40], # sneaky way to do "early stopping" 
#                        # we'll take either iter 20 or iter 40 in training process, by best valid performance
#     )

In [66]:
# lasso_searcher = sklearn.model_selection.GridSearchCV(
#     lasso,
#     lasso_hyperparameter_grid_by_name,
#     scoring='balanced_accuracy',
#     cv=cv_splitter,
#     return_train_score=True,
#     refit=False)

In [67]:
# lasso_searcher.fit(x_train_text_count, y_train)

In [68]:
# lasso_search_results_df = pd.DataFrame(lasso_searcher.cv_results_).copy()

In [69]:
# param_keys = ['param_C', 'param_max_iter']

# # Rearrange row order so it is easy to skim
# lasso_search_results_df.sort_values(param_keys, inplace=True)

In [70]:
# lasso_search_results_df[param_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score']]

In [71]:
# best_lasso = lasso.set_params(**lasso_searcher.best_params_)
# best_lasso.fit(x_train_text_count, y_train)

In [72]:
# best_lasso_yhat_test = best_lasso.predict_proba(x_test_text_count)[:,1]
# np.savetxt('yproba1_test.txt', best_lasso_yhat_test)